In [0]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("infer5chema", "true")\
.load("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv")
df.printSchema( )
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
# 데이터 타입 변환
# lit
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

Out[2]: DataFrame[5: int, five: string, 5.0: double]

In [0]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365)\
    .select("InvoiceNo", "Description")\
    .show(5)

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows



In [0]:
df.where("InvoiceNo = 536365").show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [0]:
# and / or

from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [0]:
# bool을 화용해 데이터 프레임 필터링 하기

from pyspark.sql.functions import instr

# 실제 로우를 보여주는 것이 아니면, 연산이 들어가지 않음
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
    .where("isExpensive")\
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [0]:
# .where("isExpensive") : "isExpensive" column이 true인 것만!

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|     2.55|      false|
|     3.39|      false|
|     2.75|      false|
|     3.39|      false|
|     3.39|      false|
+---------+-----------+
only showing top 5 rows



In [0]:
# spark sql을 사용해도 성능 저하 없음 engine에서 해석하기 때문
# filter는 spark sql을 사용하는 것이 더 쉬움
from pyspark.sql.functions import expr

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
    .where("isExpensive")\
    .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [0]:
# null data

df.where(col("Description").eqNullSafe("Hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [0]:
# pow 거듭 제곱

from pyspark.sql.functions import expr, pow

fabricateQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricateQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [0]:
df.selectExpr(
    "CustomerID",
    "POWER((Quantity * UnitPrice), 2) + 5"
).show(2)

+----------+--------------------------------------+
|CustomerID|(POWER((Quantity * UnitPrice), 2) + 5)|
+----------+--------------------------------------+
|   17850.0|                    239.08999999999997|
|   17850.0|                              418.7156|
+----------+--------------------------------------+
only showing top 2 rows



In [0]:
# 반올림 내림
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [0]:
# 상관관계

from pyspark.sql.functions import corr

df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [0]:
# 통계 요약

df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

In [0]:
# 하나씩 넣고 싶다면
from pyspark.sql.functions import count, mean, stddev_pop, min, max

df.select(max("Quantity")).show()

+-------------+
|max(Quantity)|
+-------------+
|           96|
+-------------+



In [0]:
# StatFunctions 패키지 : 다양한 통계 함수 제공

# approxQuantile : 데이터의 백분위수의 정확한 값 혹은 근사치를 제공

df.stat.approxQuantile("UnitPrice", [0.5, 0.75], 0.05)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-3693574929347234> in <cell line: 5>()
      3 # approxQuantile : 데이터의 백분위수의 정확한 값 혹은 근사치를 제공
      4 
----> 5 df.stat.approxQuantile("UnitPrice", [0.5, 0.75], 0.05)

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in approxQuantile(self, col, probabilities, relativeError)
   3793         relativeError: float,
   3794     ) -> Union[List[float], List[List[float]]]:
-> 3795         return self.df.approxQuantile(col, probabilities, relativeError)
   3796 
   3797     appro

In [0]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
df2 = df.withColumn("int_unit_price", 
                  df["UnitPrice"]
                  .cast('float'))
df2.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- int_unit_price: float (nullable = true)



In [0]:
df2.stat.approxQuantile("int_unit_price", [0.5, 0.75], 0.05)

Out[51]: [2.509999990463257, 4.210000038146973]

In [0]:
df.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [0]:
df.stat.freqItems(["StockCode", "Quantity"]).show(5)

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[-4, 600, 30, 2, ...|
+--------------------+--------------------+



In [0]:
# auto incresement 추가

from pyspark.sql.functions import monotonically_increasing_id

df.selectExpr("monotonically_increasing_id()",
              "Quantity").show(5)

+-----------------------------+--------+
|monotonically_increasing_id()|Quantity|
+-----------------------------+--------+
|                            0|       6|
|                            1|       6|
|                            2|       8|
|                            3|       6|
|                            4|       6|
+-----------------------------+--------+
only showing top 5 rows



In [0]:
df.select(monotonically_increasing_id().alias("id"),
          col("Quantity")
         ).show(5)

# rand() : 불규칙적인 데이터 생성
# 다양한 함수가 존재

+---+--------+
| id|Quantity|
+---+--------+
|  0|       6|
|  1|       6|
|  2|       8|
|  3|       6|
|  4|       6|
+---+--------+
only showing top 5 rows



In [0]:
# initcap 함수 : 공백으로 나뉘는 단어의 첫글자를 대문자로 변환, 나머지는 소문자로
# lower(), upper()
# lpad, ltrim, rpad, rtrim, trim 등 존재

from pyspark.sql.functions import initcap

df.select(col("Description"), initcap(col("Description"))).show(3)

+--------------------+--------------------+
|         Description|initcap(Description)|
+--------------------+--------------------+
|WHITE HANGING HEA...|White Hanging Hea...|
| WHITE METAL LANTERN| White Metal Lantern|
|CREAM CUPID HEART...|Cream Cupid Heart...|
+--------------------+--------------------+
only showing top 3 rows



In [0]:
# 정규표현식

from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GREEN|BLUE"

df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
    col("Description")
).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [0]:
# translate 함수 : 문자 치환

from pyspark.sql.functions import translate

df.select(translate(col("Description"), "HANG", "1337"),
         col("Description")
         ).show(2)

+----------------------------------+--------------------+
|translate(Description, HANG, 1337)|         Description|
+----------------------------------+--------------------+
|              W1ITE 1337I37 1E3...|WHITE HANGING HEA...|
|               W1ITE MET3L L33TER3| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [0]:
# 처음 나타난 단어를 추출 : regexp_extract

from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"

df.select(
    regexp_extract(col("Description"), extract_str, 1),
    col("Description")
).show(2)

+------------------------------------------------------------+--------------------+
|regexp_extract(Description, (BLACK|WHITE|RED|GREEN|BLUE), 1)|         Description|
+------------------------------------------------------------+--------------------+
|                                                       WHITE|WHITE HANGING HEA...|
|                                                       WHITE| WHITE METAL LANTERN|
+------------------------------------------------------------+--------------------+
only showing top 2 rows



In [0]:
# 값이 있는지 : instr

from pyspark.sql.functions import instr

containsBlack = instr(col("Description"), "BLACK") >= 1

df.withColumn("hasSimpleColor", containsBlack).where("hasSimpleColor").select("*").show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|hasSimpleColor|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+--------------+
|   536378|   85099C|JUMBO  BAG BAROQU...|      10|2010-12-01 09:37:00|     1.95|   14688.0|United Kingdom|          true|
|   536381|  15056BL|EDWARDIAN PARASOL...|       2|2010-12-01 09:41:00|     5.95|   15311.0|United Kingdom|          true|
|   536384|    82484|WOOD BLACK BOARD ...|       3|2010-12-01 09:53:00|     6.45|   18074.0|United Kingdom|          true|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+--------------+
only showing top 3 rows



In [0]:
df.select("*").where(containsBlack).show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536378|   85099C|JUMBO  BAG BAROQU...|      10|2010-12-01 09:37:00|     1.95|   14688.0|United Kingdom|
|   536381|  15056BL|EDWARDIAN PARASOL...|       2|2010-12-01 09:41:00|     5.95|   15311.0|United Kingdom|
|   536384|    82484|WOOD BLACK BOARD ...|       3|2010-12-01 09:53:00|     6.45|   18074.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



In [0]:
# 동적으로 변하는 인수의 개수

from pyspark.sql.functions import expr, locate

simpleColors = ["black", "white", "red", "green", "blue"]

def color_locator(col, col_string):
    return locate(col_string.upper(), col)\
                .cast("boolean")\
                .alias("is_" + col_string)
        
        
selectedcol = [color_locator(df.Description, c) for c in simpleColors]
#selectedcol.append(expr("*"))

print(selectedcol)

[Column<'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS is_black'>, Column<'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS is_white'>, Column<'CAST(locate(RED, Description, 1) AS BOOLEAN) AS is_red'>, Column<'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS is_green'>, Column<'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS is_blue'>]


In [0]:
selectedcol.append(expr("*"))

print(selectedcol)

[Column<'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS is_black'>, Column<'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS is_white'>, Column<'CAST(locate(RED, Description, 1) AS BOOLEAN) AS is_red'>, Column<'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS is_green'>, Column<'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS is_blue'>, Column<'unresolvedstar()'>]


In [0]:
df.select(*selectedcol).where(expr("is_white OR is_red"))\
    .select("Description").show(3)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 3 rows



In [0]:
# 오늘 날짜와 현재 타임스탬프

from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10)\
    .withColumn("today", current_date())\
    .withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")

dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [0]:
# 오늘을 기준으로 5일 전후

from pyspark.sql.functions import date_add, date_sub

dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2023-01-30|        2023-02-09|
+------------------+------------------+
only showing top 1 row



In [0]:
# 두 날짜의 차이 : datediff
# 두 날짜의 개월 수 : month_between
# to_date : 문자열을 날짜로

from pyspark.sql.functions import to_date, lit

spark.range(5).withColumn("date", lit("2017-01-01"))\
    .select(to_date(col("date"))).show(1)

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
+-------------+
only showing top 1 row



In [0]:
dateDF.select(to_date(lit("2020-20-12"))).show(1)

+-------------------+
|to_date(2020-20-12)|
+-------------------+
|               null|
+-------------------+
only showing top 1 row



In [0]:
dateDF.select(to_date(lit("2020-20-12"), "yyyy-dd-MM")).show(1)

+-------------------------------+
|to_date(2020-20-12, yyyy-dd-MM)|
+-------------------------------+
|                     2020-12-20|
+-------------------------------+
only showing top 1 row



In [0]:
from pyspark.sql.functions import to_timestamp

dateDF.select(to_timestamp(lit("2020-12-21"))).show(1)

+------------------------+
|to_timestamp(2020-12-21)|
+------------------------+
|     2020-12-21 00:00:00|
+------------------------+
only showing top 1 row



In [0]:
dateDF.filter(to_timestamp("today") > lit("2020-12-01")).show(1)

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2023-02-04|2023-02-04 02:30:...|
+---+----------+--------------------+
only showing top 1 row



In [0]:
# null : 암시적인 값보다 명시적인 값이 좋음
# nullable이 아니어도, null값을 넣을 수 있음!


# coalesce : null이 아닌 첫번째 값 반환
# ifnull : 첫번째 값이 null이면 두번째 값 반환
# nullif : 두 값이 같으면 null을 반환
# nvl    : 첫번째 값이 null이면 두번째 값 반환
# nvl2   : 첫번째 값이 null이 아니면, 두번째 값 반환
#        : 첫번째 값이 null이면, 세번째 값 반환

# drop   : null값을 가진 모든 row 제거
df.na.drop()

Out[117]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [0]:
# any : col 중 하나라도 null이면 제거
# all : 모든 col이 null이거나 NaN인 경우 제거
df.na.drop("any")

Out[119]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [0]:
# fill : null 값을 다른 값으로 채움

df.na.fill("null값 입니다")

# 적용하고자 하는 칼럼 선택

df.na.fill("null값입니다", subset=["StockCode", "InvoiceNo"])

Out[122]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [0]:
# dictionary로 칼럼에 따라 fillna 다르게 설정

fill_dict = {"StockCode" : 5, "Description" : "No Value"}
df.na.fill(fill_dict)

Out[123]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [0]:
# replace ==> 변경하고자 하는 데이터 타입이 원본과 같아야 함

df.na.replace([""], ["UNKNOWN"], "Description")

Out[124]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [0]:
# 복합 데이터 타입 : struct, array, map

from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.show(1)

+--------------------+
|             complex|
+--------------------+
|{WHITE HANGING HE...|
+--------------------+
only showing top 1 row



In [0]:
complexDF.select("complex.Description").show(2)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows



In [0]:
complexDF.select("complex.*").show(3)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
+--------------------+---------+
only showing top 3 rows



In [0]:
# split => split 후 배열로 반환

from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [0]:
df.select(split(col("Description"), " ").alias("array"))\
    .selectExpr("array[0]").show(2)

+--------+
|array[0]|
+--------+
|   WHITE|
|   WHITE|
+--------+
only showing top 2 rows



In [0]:
# 배열의 길이 조회
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



In [0]:
# 배열에 값이 있는지 확인
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



In [0]:
# explode : 배열을 행으로 변환

from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded").show(10)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
| WHITE METAL LANTERN|   536365|   WHITE|
| WHITE METAL LANTERN|   536365|   METAL|
| WHITE METAL LANTERN|   536365| LANTERN|
|CREAM CUPID HEART...|   536365|   CREAM|
|CREAM CUPID HEART...|   536365|   CUPID|
+--------------------+---------+--------+
only showing top 10 rows



In [0]:
# map

from pyspark.sql.functions import create_map

df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(2, False)

+----------------------------------------------+
|complex_map                                   |
+----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365}|
|{WHITE METAL LANTERN -> 536365}               |
+----------------------------------------------+
only showing top 2 rows



In [0]:
map_df = df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))

map_df.selectExpr("complex_map['WHITE HANGING HEART T-LIGHT HOLDER']").show(2)

+-----------------------------------------------+
|complex_map[WHITE HANGING HEART T-LIGHT HOLDER]|
+-----------------------------------------------+
|                                         536365|
|                                           null|
+-----------------------------------------------+
only showing top 2 rows



In [0]:
map_df.selectExpr("explode(complex_map)").show(5)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
|CREAM CUPID HEART...|536365|
|KNITTED UNION FLA...|536365|
|RED WOOLLY HOTTIE...|536365|
+--------------------+------+
only showing top 5 rows



In [0]:
# Json 다루기(문자열 형태로)

json_df = spark.range(1).selectExpr("""
    '{"key" : {"value" : [1, 2, 3]}}' as json_string""")

In [0]:
from pyspark.sql.functions import get_json_object, json_tuple


json_df.select(
    get_json_object(col("json_string"), "$.key.value[1]").alias("column"),
    json_tuple(col("json_string"), "key")
).show(5)

+------+-----------------+
|column|               c0|
+------+-----------------+
|     2|{"value":[1,2,3]}|
+------+-----------------+



In [0]:
# to_json : strctType to json 문자열

from pyspark.sql.functions import to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")\
    .select(to_json(col("myStruct"))).show(2, False)

+-------------------------------------------------------------------------+
|to_json(myStruct)                                                        |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+-------------------------------------------------------------------------+
only showing top 2 rows



In [0]:
# from_json : json문자열을 다시 객체로 반환
# 이때 스키마를 지정해줘야 함

from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType((
    StructField("InvoiceNo", StringType(), True),
    StructField("Description", StringType(), True)
))


df.selectExpr("(InvoiceNo, Description) as myStruct")\
    .select(to_json(col("myStruct")).alias("newJson"))\
    .select(from_json(col("newJson"), parseSchema), col("newJson")).show(2, False)

+--------------------------------------------+-------------------------------------------------------------------------+
|from_json(newJson)                          |newJson                                                                  |
+--------------------------------------------+-------------------------------------------------------------------------+
|{536365, WHITE HANGING HEART T-LIGHT HOLDER}|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{536365, WHITE METAL LANTERN}               |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+--------------------------------------------+-------------------------------------------------------------------------+
only showing top 2 rows



In [0]:
# udf

udfExampleDF = spark.range(5).toDF("num")

def power3(value):
    return value ** 3
power3(2)

Out[177]: 8

In [0]:
# 파이썬에서 udf

# 1. 함수 직렬화 후 워커에 전달
# 2. 스파크에서 파이썬 프로세스 실행 후 데이터 전송
# 3. 파이썬에서 처리 후 결과 반환

# JVM과 파이썬이 메모리 경합을 할 수 있어 스칼라나 자바로 UDF를 짜는 것이 좋음

from pyspark.sql.functions import udf

power3udf = udf(power3)

In [0]:
udfExampleDF.select(power3udf(col("num"))).show(5)

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



In [0]:
# 스칼라 코드로 작성된 udf sql함수로 등록하여 파이썬 코드에서 우회적으로 사용 가능
# spark.udf.register("power3", power3(_:Double):Double)
# 마찬가지로 hiveudf도 사용 가능

udfExampleDF.selectExpr("power3scala(num)").show(2)